In [16]:
%load_ext autoreload
%autoreload 2
import parse_agent,pde_descriptions,system_prompt
import os
from dotenv import load_dotenv
from google import genai
from google.genai.types import GenerateContentConfig
import markdown_to_json
import json

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")


fk_description = pde_descriptions.fk_description
tau_d = 0.5714
pde_description = fk_description.format(tau_d=tau_d)

parse_prompt = parse_agent.parse_prompt
system_prompt = system_prompt.system_prompt

user_prompt = parse_prompt.format(
    user_input=pde_description,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.5-pro", 
    config=GenerateContentConfig(
            system_instruction=system_prompt
        ),
    contents=user_prompt    
    )

print(response.text)


```json
{
  "pde_description": "\\begin{cases} \\partial_t u = D \\nabla^2 u - \\frac{I_{fi}(u, v) + I_{so}(u) + I_{si}(u, w)}{C_m} \\\\ \\partial_t v = (1-H(u-V_c))\\frac{1-v}{\\tau_{mv}(u)} + H(u-V_c)\\frac{-v}{\\tau_{pv}} \\\\ \\partial_t w = (1-H(u-V_c))\\frac{1-w}{\\tau_{mw}} + H(u-V_c)\\frac{-w}{\\tau_{pw}} \\end{cases} \\\\ \\text{where } H(\\cdot) \\text{ is the Heaviside step function and:} \\\\ I_{fi}(u, v) = \\frac{-v H(u - V_c) (u - V_c) (1 - u)}{\\tau_d} \\\\ I_{so}(u) = \\frac{u(1 - H(u - V_c))}{\\tau_0} + \\frac{H(u - V_c)}{\\tau_r} \\\\ I_{si}(u, w) = \\frac{-w(1 + \\tanh(k(u - V_{csi})))}{2\\tau_{si}} \\\\ \\tau_{mv}(u) = (1-H(u-V_v))\\tau_{v1} + H(u-V_v)\\tau_{v2}",
  "number_of_state_variables": 3,
  "texture_size": 512,
  "spatial_step": 0.0390625,
  "domain_size": 20.0,
  "temporal_step": 0.025,
  "time_horizon": 100.0,
  "boundary_conditions": "Neumann (No-Flux)",
  "initial_conditions": {
    "u": "Provided as input data",
    "v": "Provided as input data",
    "

In [15]:
# Simple one-liner to save the response
with open("resp.json", "w", encoding="utf-8") as f:
    f.write(response.text)